# Multi-atlas Segmentation

In [1]:
from os import listdir
from os.path import isdir, join
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from numpy.linalg import inv, det, norm
from math import sqrt, pi
from functools import partial

%matplotlib inline

/home/okozyn/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Import Helper Functions

In [2]:
%run "./src/nib-tools.py"
%run "./src/file-tools.py"
%run "./src/em.py"

## Find Weights from Mutual Information

In [3]:
# Path to test image
valid_img_path = "./registered-data/Par0009-matched-to-mni/Validation_Set/templates/"
gt_path = "./registered-data/Par0009-matched-to-mni/Validation_Set/labels/"
train_labels_path = "./registered-data/Par0009-matched-to-mni/Training_Set/labels/"
train_img_path = "./registered-data/Par0009-matched-to-mni/Training_Set/templates/"

train_dirs = [f for f in listdir(train_img_path) if isdir(join(train_img_path, f))]
valid_dirs = [f for f in listdir(valid_img_path) if isdir(join(valid_img_path, f))]

# read training labels
train_labels = []
for f in train_dirs:
    labels, _ = read_im(join(train_labels_path, f, 'result.nii.gz'))
    train_labels.append(labels)

print(train_dirs)
print(valid_dirs)

['IBSR_18', 'IBSR_03', 'IBSR_01', 'IBSR_06', 'IBSR_09', 'IBSR_07', 'IBSR_16', 'IBSR_08', 'IBSR_05', 'IBSR_04']
['IBSR_13', 'IBSR_12', 'IBSR_17', 'IBSR_11', 'IBSR_14']


In [4]:
def get_sim_metrics(train_dir_path):
    """
    Get similarity metrics from elastix log files.
    
    Args:
        train_dir_path (str): the path to a folder with registration results.
    
    Returns:
        (numpy.ndarray): similarity metrics.
    """
    import re
    
    dirs = tuple(f for f in listdir(train_dir_path) if isdir(join(train_dir_path, f)))
    sim_metrics = np.zeros(len(dirs))
    
    for index, train_dir in enumerate(dirs):
        elastix_log_file = open(join(train_dir_path, train_dir, 'elastix.log'), "r")
        elastix_log = elastix_log_file.read()
        elastix_log_file.close()
        reg_expr = r'^Final metric value\s+=\s+([\d\.\-]+)$'
        
        matches = re.findall(reg_expr, elastix_log, re.M)
        
        # 0th value - MI after affine transformation, 1st value - MI after B-splines transformation
        sim_metrics[index] = matches[1]
        
    return sim_metrics


def majority_vote_masks(train_labels, weights=None):
    """
    Get segmentation masks for CSF, GM, WM tissues.
    
    Args:
        train_labels ([numpy.ndarray]): an iterable of training labels (ground-truth segmentations).
        weights ([float]): an iterable with weights for each label, should have the same length
            as train_labels, optional.
    
    Returns:
        csf_mask, gm_mask, wm_mask (tuple(numpy.ndarray)): a tuple of masks for tissue segmentation.
    """
    csf_mask = None
    gm_mask = None
    wm_mask = None
    
    if weights is None:
        weights = np.ones(len(train_labels))

    for labels, w in zip(train_labels, weights):
        csf_vote = np.zeros_like(labels, dtype=float)
        gm_vote = np.zeros_like(labels, dtype=float)
        wm_vote = np.zeros_like(labels, dtype=float)

        csf_vote[labels == CSF_label] = w
        gm_vote[labels == GM_label] = w
        wm_vote[labels == WM_label] = w

        if (csf_mask is None) and (gm_mask is None) and (wm_mask is None):
            csf_mask = csf_vote
            gm_mask = gm_vote
            wm_mask = wm_vote
        else:
            csf_mask += csf_vote
            gm_mask += gm_vote
            wm_mask += wm_vote
    
    return csf_mask, gm_mask, wm_mask

In [14]:
sim_metrics = get_sim_metrics(train_img_path)
sim_metrics = -1 * sim_metrics
maj_vote_csf_mask, maj_vote_gm_mask, maj_vote_wm_mask = majority_vote_masks(train_labels)#, weights=sim_metrics)

CSF_atlas = normalize(maj_vote_csf_mask, max_val=1.0)
GM_atlas = normalize(maj_vote_gm_mask, max_val=1.0)
WM_atlas = normalize(maj_vote_wm_mask, max_val=1.0)

## EM+Multi-atlas

In [15]:
max_intens = 255

MAX_STEPS = 30
min_change = 0.01

test_img_path = valid_img_path

In [16]:
# Path to atlas image folder
atlas_path = "./atlases/Par0009-no-matching/"
result_path = "./segmentation-results/Par0009-no-matching/em-segmentation/"
out_dice_path = './dice-results/Par0009_no_matching_em_dice.csv'
#os.mkdir(result_path)

onlydirs = [f for f in listdir(test_img_path) if isdir(join(test_img_path, f))]

all_dice = np.zeros((len(onlydirs),3))

GM_atlas_flat = GM_atlas.flatten()
WM_atlas_flat = WM_atlas.flatten()
CSF_atlas_flat = CSF_atlas.flatten()
mask = GM_atlas + WM_atlas + CSF_atlas
mask_data = mask.copy().flatten()
mask_data = np.transpose(mask_data)

for i, f in enumerate(onlydirs):
    # Load all data for EM algorithm
    test_data, test_img = read_im(join(test_img_path, f, 'result.1.nii.gz'))
    test_data = normalize(test_data, max_intens)

    _, groundtruth_img = read_im(join(gt_path, f, "result.nii.gz"))

    # Pre-process feature vector to remove background points from algorithm
    # and save those indicies to add back
    features = test_data.copy().flatten()
    features = np.transpose(features)   

    features_nonzero_row_indicies = np.nonzero(mask_data)
    features_nonzero = features[features_nonzero_row_indicies]

    # row index shifted by +1 will correspond to tissue labels from ground-truth
    features_nonzero_pred = np.array((CSF_atlas_flat[features_nonzero_row_indicies],
                                      WM_atlas_flat[features_nonzero_row_indicies],
                                      GM_atlas_flat[features_nonzero_row_indicies]))
    y_pred = np.argmax(features_nonzero_pred, axis=0)

    # intialize EM algorithm
    class0 = features_nonzero[np.argwhere(y_pred == 0)[:,0]]
    class1 = features_nonzero[np.argwhere(y_pred == 1)[:,0]]
    class2 = features_nonzero[np.argwhere(y_pred == 2)[:,0]]

    # Compute mean and variance of each class
    mean0 = np.mean(class0, axis = 0)
    mean1 = np.mean(class1, axis = 0)
    mean2 = np.mean(class2, axis = 0)
    cov0 = np.cov(class0, rowvar = False)
    cov1 = np.cov(class1, rowvar = False)
    cov2 = np.cov(class2, rowvar = False)

    # Class distribution
    a0 = class0.shape[0] / features_nonzero.shape[0]
    a1 = class1.shape[0] / features_nonzero.shape[0]
    a2 = class2.shape[0] / features_nonzero.shape[0]

    # Compute Gaussian mixture model for each point
    p0 = gaussian_mixture(features_nonzero,  mean = mean0, cov = cov0)
    p1 = gaussian_mixture(features_nonzero,  mean = mean1, cov = cov1)
    p2 = gaussian_mixture(features_nonzero,  mean = mean2, cov = cov2)

    # # Compute membership weight for each point
    weights = membership_weight(p0, p1, p2, a0, a1, a2)
    # get initial log-likelihood
    log_likelihood = get_log_likelihood((a0, a1, a2), (p0, p1, p2))

    n_steps = 0

    while True:
        # Maximization step: Use that classification to reestimate the parameters
        # Class distribution
        counts = np.sum(weights, axis=0)

        a0 = counts[0] / len(features_nonzero)
        a1 = counts[1] / len(features_nonzero)
        a2 = counts[2] / len(features_nonzero)

        # Calculate mean and covariance for new classes
        mean0 = (1/counts[0]) * (weights[:, 0] @ features_nonzero)
        mean1 = (1/counts[1]) * (weights[:, 1] @ features_nonzero)
        mean2 = (1/counts[2]) * (weights[:, 2] @ features_nonzero)
        cov0 = (1/counts[0]) * ((weights[:, 0] * (features_nonzero - mean0)) @ (features_nonzero - mean0))
        cov1 = (1/counts[1]) * ((weights[:, 1] * (features_nonzero - mean1)) @ (features_nonzero - mean1))
        cov2 = (1/counts[2]) * ((weights[:, 2] * (features_nonzero - mean2)) @ (features_nonzero - mean2))

        p0 = gaussian_mixture(features_nonzero,  mean = mean0, cov = cov0)
        p1 = gaussian_mixture(features_nonzero,  mean = mean1, cov = cov1)
        p2 = gaussian_mixture(features_nonzero,  mean = mean2, cov = cov2)

        # Compute membership weight for each point
        weights = membership_weight(p0, p1, p2, a0, a1, a2)

        log_likelihood_new = get_log_likelihood((a0, a1, a2), (p0, p1, p2))

        dist_change = abs((log_likelihood_new - log_likelihood) / log_likelihood)
        print("********************************************************************")
        print(f"Img {f}")
        print("Step %d" % n_steps)
        print("Distribution change %f" % dist_change)
        print(a0, a1, a2)

        n_steps += 1

        # check whether we reached desired precision or max number of steps
        if (n_steps >= MAX_STEPS) or (dist_change <= min_change):
            print("Loop stopped")
            break
        else:
            log_likelihood = log_likelihood_new

    y_pred = np.argmax(weights, axis=1)
    segment_nii_atlas = integrate_atlas_nii(test_img, y_pred, features_nonzero, 
                               features_nonzero_row_indicies, weights, CSF_atlas, 
                               GM_atlas, WM_atlas)

    # Calculate DICE
    all_dice[i,0], all_dice[i,1], all_dice[i,2] = dice_similarity(segment_nii_atlas, groundtruth_img)

    # Make directory to save result seg
   # new_dir = join(result_path,f)
    #os.mkdir(new_dir)
    #nib.save(segment_nii_atlas, join(new_dir,'atlas_EM_seg.nii.gz'))

save_dice(out_dice_path, onlydirs, all_dice)
# print(np.mean(all_dice, axis=0))
# all_dice

********************************************************************
Img IBSR_13
Step 0
Distribution change 0.025566
0.013099336726753747 0.4504321428358337 0.5364685204403042
********************************************************************
Img IBSR_13
Step 1
Distribution change 0.016794
0.018664028062159176 0.5770037502991612 0.40433222163661403
********************************************************************
Img IBSR_13
Step 2
Distribution change 0.012760
0.025198431441216203 0.6838181712308438 0.29098339732916906
********************************************************************
Img IBSR_13
Step 3
Distribution change 0.008638
0.031220877090404168 0.7614755062031549 0.20730361670700584
Loop stopped
********************************************************************
Img IBSR_12
Step 0
Distribution change 0.024910
0.009691647678175841 0.471273627112572 0.5190347252069643
********************************************************************
Img IBSR_12
Step 1
Distribution chan

In [13]:
# experiments with weighted multi-atlas
print(all_dice)
print(np.mean(all_dice, axis=0))

[[0.73815124 0.8157114  0.80473421]
 [0.85011393 0.78033749 0.82912281]
 [0.86132058 0.79138092 0.80600781]
 [0.86190173 0.76952285 0.82103951]
 [0.79405735 0.80653402 0.82950308]]
[0.82110897 0.79269734 0.81808148]


In [17]:
# experiments with multi-atlas
print(all_dice)
print(np.mean(all_dice, axis=0))

[[0.72695911 0.80754481 0.81038132]
 [0.84551396 0.77880952 0.82519491]
 [0.86380167 0.82515532 0.8185299 ]
 [0.86042216 0.76879285 0.81650924]
 [0.82544768 0.80115997 0.82965957]]
[0.82442892 0.7962925  0.82005499]
